## Практика по теме "Деревья решений"

sklearn - популярная библиотека, которая содержит основные модели машинного обучения и датасеты.  

In [265]:
from sklearn import datasets

dataset = datasets.load_iris()
print(dataset.feature_names)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


Как следует из описания (`dataset['DESCR']`), датасет содержит 150 записей, которые описывают 3 вида ирисов (щетинистый, разноцветный, виргинский) по длине и ширине их лепестков и чашелистников.  
Итак мы получили набор данных с 4мя фичами и 3мя классами

Обычно все размеченные данные разбиваются на 3 части: обучающая выборка, валидационая и тестовая. Во время экспериментов с моделями тестовая выборка не используется. Все параметры настраиваются по результатам метрик на валидационной выборке. А финальные результаты демострируются на тестовой.

In [266]:
from sklearn.model_selection import train_test_split

X = dataset.data
y = dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)


['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


Чтобы нам как-то измерить результаты модели, реализуем метрику accuracy разделив верно предсказанные классы на размер выборки

In [139]:
def score(y_true, y_pred):
    return sum(y_true == y_pred) / len(y_true)

Попробуем подобрать глубину дерева с помощью кросс-валидации

In [140]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

for max_depth in [1, 2, 3, None]:
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=17)
    scores = cross_val_score(model, X_train, y_train, cv=3, error_score=score)
    print(max_depth, scores.mean(), scores)

1 0.7047619047619048 [0.68571429 0.71428571 0.71428571]
2 0.9428571428571427 [0.91428571 1.         0.91428571]
3 0.9333333333333332 [0.91428571 1.         0.88571429]
None 0.9238095238095237 [0.91428571 0.97142857 0.88571429]


Если мы будем перебирать все параметры, которые есть у дерева, то это может занять достаточно много времени, тем более если мы хотим это распаралеллить. Поэтому в библиотеке sklearn уже есть готовый метод для перебора параметров. Воспользуемся, например, `GridSearchCV`.

In [141]:
from sklearn.model_selection import GridSearchCV

g = GridSearchCV(estimator=DecisionTreeClassifier(random_state=17),
                 param_grid={'max_depth': list(range(1, 5)) + [None],
                             'max_features': ['sqrt', 'log2', None],
                             'class_weight': [None, 'balanced']
                             },
                 error_score=score, cv=3, n_jobs=4)
g.fit(X_train, y_train)
print(f'Best score: {g.best_score_}')
print(f'Best params: {g.best_params_}')

Best score: 0.9428571428571427
Best params: {'class_weight': None, 'max_depth': 2, 'max_features': None}


После того, как нашли лучшие параметры, давайте посмотри точность модели на отложенной выборке.

In [142]:
model = DecisionTreeClassifier(class_weight=None, max_depth=2, max_features=None, random_state=17)
model.fit(X_train, y_train)
print(f'Train score: {score(y_train, model.predict(X_train))}')

pred = model.predict(X_test)
print(f'Test score: {score(y_test, pred)}')

Train score: 0.9523809523809523
Test score: 0.9777777777777777


In [143]:
model.predict(X_test)

array([0, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 0, 1, 0, 2, 0, 0, 2, 2, 2, 1, 0,
       2, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 2, 1, 0, 1, 1, 0, 1,
       2])

sklearn использует методы одного вида для каждой из своих моделей. Эти методы стали стандартом в индустрии.
Вот основные методы:
- `fit(X_train, y_train)` тут мы обучаем нашу модель.
- `predict_proba(X_test)` здесь мы предскажем вероятность отнесения к каждому классу элементов теста. Вероятность для каждой модели рассчитывается по разному. Для `DecosionTreeClassifier` - это доля элементов класса в "листе".
- `predict(X_test)` а тут мы в результате выдадим уже метки классов для теста. Часто реализация метода выглядит как argmax от predict_proba.

Отличный результат!

Теперь давайте попробуем нарисовать дерево, которое мы построили. Будем делать с помощью функционала `export_graphviz`.

In [144]:
from sklearn.tree import export_graphviz

fit_model = model.fit(X_train, y_train)
export_graphviz(fit_model, out_file='tree.dot', filled=True,
                feature_names=['sepal length', 'sepal width', 'petal length', 'petal width'])

На выходе вы получаем файл `tree.dot`, который содержит всю необходимую для нас информацию. Но для того, чтобы мы смогли посмотреть на наше дерево, нам надо перевести этот файл в картинку. Воспользуемся для этого консольной командой `dot`.

In [13]:
!dot -Tpng tree.dot -o tree.png

На выходе получаем картинку `tree.png`, которая и показана ниже.

![](tree.png)

## Классная работа № 4

В данной классной работе мы попробуем обучить несколько различных деревьев решений на другом известном датасете.

In [282]:
dataset = datasets.load_wine()

In [283]:
print(dataset['DESCR'])
print(dataset.feature_names)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

Согласно описанию датасета перед нами выборка 178 винных напитков, для которых мы будем предсказывать один из 3 классов.\
Для начала разделим выборку на обучающую и отложенную.

In [284]:
from sklearn.model_selection import train_test_split

X = dataset.data
y = dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

Ссылка на Google форму для проверки ответов: https://forms.gle/jDNb2iDNZSUTi6Fx6

**1. Обучите на имеющейся выборке дерево решений (DecisionTreeClassifier) максимальной глубины 2. Найдите точность (определяется с помощью функции `score`) у этого дерева.**\
Используйте параметр `random_state=17` для воспроизводимости результатов.

In [285]:

model = DecisionTreeClassifier(max_depth=2, random_state=17)
scores = cross_val_score(model, X_train, y_train, cv=3, error_score=score)
print(scores.mean(), scores)

0.8387533875338753 [0.83333333 0.85365854 0.82926829]


**2. С помощью `GridSearchCV` найдите наилучшие параметры из перечисленных ниже.**\
Используйте параметр `random_state=17` для воспроизводимости результатов.

In [286]:
# tree params for grid search
tree_params = {'max_depth': list(range(1, 5)), 
               'min_samples_leaf': list(range(1, 5))}

from sklearn.model_selection import GridSearchCV

g_search_cv_task2 = GridSearchCV(estimator=DecisionTreeClassifier(random_state=17),
                                 param_grid=tree_params,
                                 error_score=score, cv=3, n_jobs=4)               
g_search_cv_task2.fit(X_train, y_train)
print(f'Best score: {g_search_cv_task2.best_score_}')
print(f'Best params: {g_search_cv_task2.best_params_}')

Best score: 0.9274099883855982
Best params: {'max_depth': 4, 'min_samples_leaf': 1}


**3. Какой точностью обладает модель, которая была обучена в задании 2?**\
Используйте параметр `random_state=17` для воспроизводимости результатов.

In [293]:
model = DecisionTreeClassifier(max_depth=4, min_samples_leaf=1, random_state=17)
model.fit(X_train, y_train)
print(f'Train score: {score(y_train, model.predict(X_train))}')

pred = model.predict(X_test)
print(f'Test score: {score(y_test, pred)}')
model.predict(X_test)

Train score: 0.9838709677419355
Test score: 0.9444444444444444


array([1, 1, 1, 0, 2, 1, 0, 1, 2, 0, 2, 0, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1,
       0, 2, 1, 1, 0, 1, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 2, 1, 2, 1, 1, 2,
       2, 2, 1, 1, 0, 0, 2, 1, 1, 2])

**4. С помощью `GridSearchCV` найдите наилучшие параметры из перечисленных ниже. В отличие от задания 2 подбирайте параметры на всех выборке целиком (без деления на train и test)**\
Используйте параметр `random_state=17` для воспроизводимости результатов.

In [288]:
# tree params for grid search
tree_params = {'max_depth': list(range(1, 5)), 
               'min_samples_leaf': list(range(1, 5))}


from sklearn.model_selection import GridSearchCV
import numpy as np

g_search_cv_task4 = GridSearchCV(estimator=DecisionTreeClassifier(random_state=17),
                                 param_grid=tree_params,
                                 error_score=score, cv=3) 

g_search_cv_task4.fit(X, y)
print(f'Best params: {g_search_cv_task4.best_params_}')

Best params: {'max_depth': 3, 'min_samples_leaf': 4}


**5. Какой точностью обладает лучшая модель, которая была обучена в задании 4?**\
Используйте параметр `random_state=17` для воспроизводимости результатов.

In [289]:
print(f'Best score: {g_search_cv_task4.best_score_}')

Best score: 0.876647834274953


**6. Какой признак (порядковый номер) используется чаще всего деревом из задания 2?**\
Используйте параметр `random_state=17` для воспроизводимости результатов.

In [290]:
print(f'Best features: {g_search_cv_task2.best_estimator_.feature_importances_}')
print(f'Best feature: {np.argmax(g_search_cv_task2.best_estimator_.feature_importances_)}')

Best features: [0.03141605 0.01910435 0.         0.         0.         0.
 0.0717388  0.         0.         0.40845112 0.04849567 0.
 0.420794  ]
Best feature: 12


In [299]:
from sklearn.tree import export_graphviz

fit_model = model.fit(X_train, y_train)
export_graphviz(fit_model, out_file='tree_wine.dot', filled=True,
                feature_names=dataset.feature_names)


In [300]:
!dot -Tpng tree_wine.dot -o tree_wine.png

In [303]:
print(dataset.feature_names)

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
